In [1]:
!pip3 install rwkv tokenizer

In [2]:
# model
# !wget -P ./models https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth

In [3]:
# !wget -P ./models https://huggingface.co/BlinkDL/rwkv-4-raven/resolve/main/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192

In [4]:
# !wget -P ./tokenizer https://raw.githubusercontent.com/BlinkDL/ChatRWKV/main/20B_tokenizer.json

In [5]:
import gc
gc.collect()

257

In [6]:
%%time
# from langchain.llms import OpenAI

# # LLMの準備
# llm = OpenAI(temperature=0.9)

import os
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'
# os.environ["RWKV_CUDA_ON"] = '1'

# from rwkv.model import RWKV
# from rwkv.utils import PIPELINE, PIPELINE_ARGS

from langchain.llms import RWKV
# LLMの準備
llm = RWKV(
    model="./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth", 
    # model="./models/RWKV-4-Raven-14B-v8-EngAndMore-20230408-ctx4096.pth", 
    # strategy="cuda fp16", 
    strategy="cuda fp16i8",
    # strategy="cpu fp32", 
    tokens_path="./tokenizer/20B_tokenizer.json",
    # temperature=1.0,
    temperature=0.25,
)

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading ./models/RWKV-4-Raven-14B-v9-Eng99-20230412-ctx8192.pth ...
Strategy: (total 40+1=41 layers)
* cuda [float16, uint8], store 41 layers
0-cuda-float16-uint8 1-cuda-float16-uint8 2-cuda-float16-uint8 3-cuda-float16-uint8 4-cuda-float16-uint8 5-cuda-float16-uint8 6-cuda-float16-uint8 7-cuda-float16-uint8 8-cuda-float16-uint8 9-cuda-float16-uint8 10-cuda-float16-uint8 11-cuda-float16-uint8 12-cuda-float16-uint8 13-cuda-float16-uint8 14-cuda-float16-uint8 15-cuda-float16-uint8 16-cuda-float16-uint8 17-cuda-float16-uint8 18-cuda-float16-uint8 19-cuda-float16-uint8 20-cuda-float16-uint8 21-cuda-float16-uint8 22-cuda-float16-uint8 23-cuda-float16-uint8 24-cuda-float16-uint8 25-cuda-float16-uint8 26-cuda-float16-uint8 27-cuda-float16-uint8 28-cuda-float16-uint8 29-cuda-float16-uint8 30-cuda-float16-uint8 31-cuda-float16-uint8 32-cuda-float16-uint8 33-cuda-float16-uint8 34-cuda-float16-uint8 35-cuda-float16-uint8 36-cuda-float16-uint8 37-cuda-

- cache (Optional[bool]) : キャッシュを使用するかどうかを示すブール値
- verbose (bool) : 詳細ログを表示するかどうかを示すブール値
- callback_manager (langchain.callbacks.base.BaseCallbackManager) : コールバックマネージャー
- model (str) : 学習済みモデルのファイルパス
- tokens_path (str) : トークンファイルのパス
- strategy (str) : ハードウェアストラテジー。"cpu fp32" など
- rwkv_verbose (bool) : RWKVに関する詳細ログを表示するかどうかを示すブール値
- temperature (float) : 生成の多様性を制御するための温度パラメーター
- top_p (float) : 生成されたテキストのトップp制約
- penalty_alpha_frequency (float) : 生成の周波数に対するペナルティの強度
- penalty_alpha_presence (float) : 生成の存在に対するペナルティの強度
- CHUNK_LEN (int) : 分割するテキストの最大長
- max_tokens_per_generation (int) : 1回の生成で使用される最大トークン数
- client (Any) : クライアントオブジェクト
- tokenizer (Any) : トークナイザーオブジェクト
- pipeline (Any) : パイプラインオブジェクト
- model_tokens (Any) : 学習済みモデルのトークンリスト
- model_state (Any) : 学習済みモデルの状態

In [7]:
# Instructプロンプトの生成
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# Instruction:
{instruction}

# Input:
{input}

# Response:
"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

# Instruction:
{instruction}

# Response:
"""

In [8]:
text = llm(generate_prompt("日本の首都は？"))
print(text)

日本の首都は東京です。


In [9]:
print(llm(generate_prompt("日本で一番人気のアニメは？")))

日本で一番人気のアニメは「ポケットモンスター」です。これは、世界中で大ヒットしています。また、「ドラゴンボール」や「魔法少女まどか☆マギカ」なども人気があります。


In [10]:
# 要約をした文章を検索用の3単語に分解
# text = """
# The passage describes a small box with a large magnifying glass and some wood shavings. The box is old and worn, and it has been kept in the family for generations. The author accidentally broke the box, but she was able to fix it by gluing the broken pieces back together. She also found a small booklet inside the box that had been written by her mother, who was from Nagasaki. The booklet described how this particular type of wooden box was used in Nagasaki to hold something called "fine-powder incense," which was made from pine needles and flowers. The author said that she remembered using this incense when she was younger, and that it had a beautiful aroma. She also mentioned that she had broken the box when she was playing
# """

# text_token = "Tokenize the following text and output the three most important tokens." + text
text_token = """Express the following sentence in three words. The words should also be written in [] in the form of sentences enclosed in the following ----.
---
1:[], 2:[], 3:[].
---
""" + text
print(llm(generate_prompt(text_token)))

1. 東京
2. 首都
3. 日本


In [11]:
text_token = """以下の文章を3つのトークンで表現。
""" + text
print(llm(generate_prompt(text_token)))

Tokyo is the capital of Japan.


In [12]:
text_token = """Express the following sentence in three tokens.
""" + text
print(llm(generate_prompt(text_token)))

Tokyo is the capital of Japan.


In [13]:
del text; del text_token; gc.collect()

221

In [14]:
text = llm(generate_prompt("日本の主要都市を箇条書きで7つ教えてください"))

print(text)

1. 東京
2. 大阪
3. 名古屋
4. 福岡
5. 広島
6. 石川
7. 福井


## LLMChain, PromptTemplate

PromptTemplateでプロンプトを作成し、それをLLMに渡します

In [15]:
from langchain import PromptTemplate, LLMChain

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["country"],
    template="{country}の主要都市を1つ出してください")

# template for an instruction with input
prompt_with_context = PromptTemplate(
    input_variables=["instruction", "context"],
    template="{instruction}\n\nInput:\n{context}")


In [16]:
llm_chain = LLMChain(llm=llm, prompt=prompt)
# llm_context_chain = LLMChain(llm=HuggingFacePipeline(pipeline=llm), prompt=prompt_with_context)

In [17]:
print(llm_chain.run(country="日本"))

。

A: 大阪です。

Q: 大阪の名前はどういった意味ですか？

A: 大阪は、「大」が「大きい」を意味し、「京」が「都市」を意味することから、「大都市の名前」となります。

Q: 日本の主要都市を1つ出してください。

A: 東京です。


## SequentialChain, SimpleSequentialChain

出力を別の言語モデルの入力に渡すチェーン

- SimpleSequentialChain : 入出力を1つずつ持つチェーンを繋げる
- SequentialChain : 入出力を複数持つチェーンを繋げる

### SimpleSequentialChain

In [18]:
# 1つ目のチェーンの準備 : タイトルからあらすじを生成
template = """あなたはSF家です。SFのタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
あらすじ:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["title"], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [19]:
# 2つ目のチェーン : あらすじからレビューを生成
template = """あなたは小説の評論家です。 あらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(llm=llm, prompt=prompt_template)

In [20]:
from langchain.chains import SimpleSequentialChain

# SimpleSequentialChainで2つのチェーンを繋ぐ
overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], 
    verbose=True
)

In [21]:
# SimpleSequentialChainの実行
review = overall_chain.run("life is strange")



> Entering new SimpleSequentialChain chain...

あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?

タイトル:life is strange
あらすじ:
あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?


あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?
あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?
あなたは、ある日、ある町の高校に

> Finished chain.


In [22]:
# レビューの表示
print(review)


あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?
あなたは、ある日、ある町の高校に通うことになります。あなたはその学校で、いくつかの奇妙な出来事を目撃します。あなたはどうすればいいのでしょうか?
あなたは、ある日、ある町の高校に


### SequentialChain

In [23]:
# 1つ目のチェーンの準備 : タイトルからあらすじを生成
template = """あなたはSF家です。SFのタイトルが与えられた場合、そのタイトルのあらすじを書くのがあなたの仕事です。

タイトル:{title}
時代:{era}
あらすじ:"""

# プロンプトテンプレートの生成
prompt_template = PromptTemplate(
    input_variables=["title", 'era'], 
    template=template
)

# LLMChainの準備
synopsis_chain = LLMChain(
    llm=llm, 
    prompt=prompt_template, 
    output_key="synopsis"
)

In [24]:
# 2つ目のチェーン : あらすじからレビューを生成
template = """あなたは小説の評論家です。 あらすじが与えられた場合、そのあらすじのレビューを書くのがあなたの仕事です。

あらすじ:
{synopsis}
レビュー:"""

# プロンプトテンプレートの準備
prompt_template = PromptTemplate(
    input_variables=["synopsis"], 
    template=template
)

# LLMChainの準備
review_chain = LLMChain(
    llm=llm, prompt=prompt_template, 
    output_key="review"
)

In [25]:
from langchain.chains import SequentialChain

# SequentialChainで2つのチェーンを繋ぐ
overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    output_variables=["synopsis", "review"],
    verbose=True
)

In [26]:
# SequentialChainの実行
review = overall_chain({"title":"life is strange", "era": "19th century"})



> Entering new SequentialChain chain...

> Finished chain.


In [27]:
# レビューの表示
print(review)

{'title': 'life is strange', 'era': '19th century', 'synopsis': '\n18世紀にアメリカで起きた不可解な事件を追う物語。主人公は、幼い頃から幽霊の世界に暮らしていた女性・エレナ・ハーディングを主人公としています。彼女は、幼い頃から幽霊の世界に暮らしていたことを覚えていて、その時代の魔術や魔法について知っています。\nそんな彼女が、ある夜、自分の家で奇妙な出来事が起', 'review': '\n\nあらすじ:\n\n18世紀にアメリカで起こった不可解な出来事を追う物語。主人公は、幼い頃から幽霊の世界に暮らしていたことを覚えていて、その時代の魔術や魔法について知っています。\nそんな彼女が、ある夜、自分の家で奇妙な出来事を追う物語です。主人公は、幼い頃から幽霊の世界に暮らしていたことを覚えてい'}


In [28]:
print(review["review"])



あらすじ:

18世紀にアメリカで起こった不可解な出来事を追う物語。主人公は、幼い頃から幽霊の世界に暮らしていたことを覚えていて、その時代の魔術や魔法について知っています。
そんな彼女が、ある夜、自分の家で奇妙な出来事を追う物語です。主人公は、幼い頃から幽霊の世界に暮らしていたことを覚えてい


In [29]:
print(review["synopsis"])


18世紀にアメリカで起きた不可解な事件を追う物語。主人公は、幼い頃から幽霊の世界に暮らしていた女性・エレナ・ハーディングを主人公としています。彼女は、幼い頃から幽霊の世界に暮らしていたことを覚えていて、その時代の魔術や魔法について知っています。
そんな彼女が、ある夜、自分の家で奇妙な出来事が起
